# Manipulación de datos con [*pandas*](https://pandas.pydata.org/)

En este clase veremos 
- como crear dataframes a partir de datos de distintas fuentes 
- funciones avanzadas de manipulación de dataframes

In [ ]:
import pandas as pd
print("Versión de pandas "+pd.__version__)
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
from IPython.display import display

## Lectura de archivos CSV

Descargemos la base de datos "Dow Jones Index" del repositorio UCI

https://archive.ics.uci.edu/ml/datasets/Dow+Jones+Index

In [ ]:
!wget -c https://archive.ics.uci.edu/ml/machine-learning-databases/00312/dow_jones_index.zip
!unzip -o dow_jones_index.zip
!head -n 10 dow_jones_index.data
!cat dow_jones_index.names

Pandas provee la función [read_csv()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)

Algunos argumentos de interes
- sep: Caracter delimitador
- header: Número de la fila que se usara para nombrar las columnas
- index_col: Numero de la columna que se usara como índice de fila
- usecols: Lista especificando que columnas usar
- dtype: Diccionario especificando los tipos de las columnas
- converters: Diccionario de funciones para parsear datos
- skiprows: Cuantas lineas saltarse antes de empezar a leer
- na_values: Caracteres para usar como missing value

In [ ]:
df = pd.read_csv("dow_jones_index.data", header=0, index_col='stock')
df.head()

In [ ]:
df.loc["AA"].plot(x="date", y="open")

Parseando fechas y tiempos

## Lectura de archivos excel

- Pandas provee la función [read_excel()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html)

- Requisito adicional: python-xlrd

In [ ]:
!wget -c http://www.censo2017.cl/wp-content/uploads/2017/12/Cantidad-de-Viviendas-por-Tipo.xlsx

In [ ]:
df = pd.read_excel("Cantidad-de-Viviendas-por-Tipo.xlsx", sheet_name=1, 
                   usecols=list(range(1, 20)), header=1, index_col='ORDEN')
df.head()

Notemos la estructura jerárquica de la tabla: REGION, PROVINCIA, COMUNA

- Podriamos querer obtener los valores totales de la Región de los Ríos: reducción suma

In [ ]:
row_mask = df["NOMBRE REGIÓN"] == "LOS RÍOS"
col_mask = df.columns[6:-1]
display(df.loc[row_mask])
df.loc[row_mask, col_mask].sum()

## Groupby: Reducciones condicionales

En el caso anterior podriamos querer reducir de forma separada para cada provincia

La función [groupby](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) permite hacer una reducción condicional a un índice/etiqueta

Podemos imaginar que la función *groupby* es una [secuencia](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) como la siguiente:


Donde
- Split: divide los datos según una **llave**
- Apply: Realiza una función sobre cada grupo: reducción, transformación, filtrado
- Combine: Mezcla el resultado en un nuevo dataframe

In [ ]:
df.loc[row_mask].groupby("NOMBRE PROVINCIA").sum()

In [ ]:
df.loc[row_mask].groupby("NOMBRE PROVINCIA").apply(lambda x: x+1)

## Cambio de índice

Podemos usar las funciones [reset_index](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html) y [set_index](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html#pandas.DataFrame.set_index) para modificar el índice del dataframe a nuestra conveniencia

In [ ]:
# df = df.reset_index()
# df = df.set_index("NOMBRE COMUNA")
row_mask = df["NOMBRE REGIÓN"] == "LOS RÍOS"
col_mask = df.columns[6:-1]
print(col_mask)
fig, ax = plt.subplots(figsize=(8, 5), tight_layout=True)
df.loc[row_mask, col_mask].plot(ax=ax, kind='line', stacked=False, logy=True);
ax.legend(loc=2, ncol=1, bbox_to_anchor=(0, 1.5));

- Lectura y parseo de datos
- Apply
- Split/Combine
- GroupBy
- Multi índices
- Merge/Join 


## Lectura de archivos H5DF